In [2]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing
pd.options.mode.chained_assignment = None

## Задача 1. 

In [10]:
df = pd.read_csv('D:\DS\DS school\Химический анализ родника в Нескучном саду.csv', sep=';')
df

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


Примечание: при первоначальном варианте я рассматривал наличие строковых значений в столбце "Результат анализа" как ошибку,
т.к. столбец предполагает количественные ограничения (о чем говорит и "Единица измерения"), однако при проверке задания такая логика была названа ошибочной. В этот раз буду считать, что "б/цвета" считается вхождением в норматив.
Если бы у нас был датасет на тысячи строк, мы бы вряд ли смогли адекватно обработать строковые значения в нормативном столбце,
поэтому я бы все же добавил проверку на такой случай и при выявлении "строк" отмечал их "Не норма" или NaN.

In [230]:
# сделаем временные датасеты для проверки на "пределы" и "не более"

In [13]:
df_lim = df[df['Норматив'].str.contains('предел')]
df_lim['Норматив_tmp'] = df['Норматив'].str.split(' ').str[2].str.split('-')
df_lim

,Показатель,Единица измерений,Результат анализа,Норматив,Норматив_tmp
0,pH,единицы pH,8.4,в пределах 6-9,"[6, 9]"
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,"[7, 10]"


In [14]:
df_ml = df[df['Норматив'].str.contains('более')]
df_ml['Норматив_tmp'] = df['Норматив'].str.split(' ').str[2].str.replace(',', '.').str.split('-')
df_ml

,Показатель,Единица измерений,Результат анализа,Норматив,Норматив_tmp
1,Запах,баллы,1,не более 2-3,"[2, 3]"
2,Цветность,градусы,б/цвета,не более 30,[30]
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",[1.5]
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",[3.3]
6,Нитраты (по NO3),мг/дм3,24,не более 45,[45]
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5",[3.5]
8,Хлориды (Cl),мг/дм3,200,не более 350,[350]
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,[500]
10,Железо (включая хлорное железо) по Fe,мг/дм3,0.019,"не более 0,3",[0.3]
11,Нефть,мг/дм3,0.55,"не более 0,3",[0.3]


In [ ]:
# осуществим проверку на вхождение в норматив по двум временным датасетам

In [15]:
df_lim['Результат'] = np.where((df_lim['Результат анализа'].astype(float) >= df_lim['Норматив_tmp'].str[0].astype(float)) 
                               | (df_lim['Результат анализа'].astype(float) <= df_lim['Норматив_tmp'].str[1].astype(float)), 
                               'Норма', 'Не норма')

In [16]:
df_ml['Результат анализа'] = df_ml['Результат анализа'].str.replace('б/цвета', '0')

In [17]:
df_ml['Результат'] = np.where((df_ml['Результат анализа'].astype(float) <= df_ml['Норматив_tmp'].str[-1].astype(float)),
                              'Норма', 'Не норма')

In [ ]:
# собираем итоговый вариант

In [18]:
df = pd.concat([df_lim, df_ml]).sort_index().drop('Норматив_tmp', axis=1).set_index('Показатель')
df

,Единица измерений,Результат анализа,Норматив,Результат
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,Норма
Запах,баллы,1,не более 2-3,Норма
Цветность,градусы,0,не более 30,Норма
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,Норма
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",Норма
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",Норма
Нитраты (по NO3),мг/дм3,24,не более 45,Норма
Фосфаты (P),мг/дм3,0.36,"не более 3,5",Норма
Хлориды (Cl),мг/дм3,200,не более 350,Норма


## Задача 2.

Для разнообразия написал второй способ решения - через датафреймы

In [22]:
fruits, x = list(['O'] * 5 + ['A'] * 4), ['O'] * 3
cnt = 0
for i in range(10000):
    res = np.random.choice(fruits, 3, replace = False)
    if sorted(res) == x:
        cnt += 1
print(cnt/10000)

0.1181


In [21]:
df = pd.DataFrame([np.random.choice(['O'] * 5 + ['A'] * 4, 3, replace = False) for i in range(10000)])
len(df[(df[0] == 'O') & (df[1] == 'O') & (df[2] == 'O')]) / 10000

0.1199

In [ ]:
# вероятность подтвердилась экспериментально на 10 000 экспериментах

## Задача 3.

Для разнообразия написал второй способ решения - через датафреймы

In [23]:
cnt = 0
for i in range(10000):
    arr = ['S'] * 7 + ['N'] * 3
    if np.random.choice(arr, 1) == 'S':
        arr.remove('S')
        if np.random.choice(arr, 1) == 'N':
            cnt += 1
print(cnt/10000)

0.2341


In [24]:
df = pd.DataFrame([np.random.choice(['S'] * 7 + ['N'] * 3, 2, replace = False) for i in range(10000)])
len(df[(df[0] == 'N') & (df[1] == 'S')]) / 10000

0.2382

In [ ]:
# вероятность подтвердилась экспериментально на 10 000 экспериментах